In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import pickle
import ujson
import sys
import os
import csv

import pandas as pd
import numpy as np

from tqdm import tqdm
from collections import defaultdict
from typing import Optional, Union

from bigbio.dataloader import BigBioConfigHelpers

sys.path.append('../../../..')
sys.path.append('..')
from DataModule import process_mention_dataset, process_umls_ontology, process_obo_ontology
from umls_utils import UmlsMappings
from bigbio_utils import CUIS_TO_REMAP, CUIS_TO_EXCLUDE, DATASET_NAMES, VALIDATION_DOCUMENT_IDS
from bigbio_utils import dataset_to_documents, dataset_to_df, resolve_abbreviation, get_left_context, get_right_context
from bioel.ontology import BiomedicalOntology

conhelps = BigBioConfigHelpers()


/nethome/cye73/conda_envs/arboel_2/lib/python3.9/site-packages/datasets/load.py:923: FutureWarning: The repository for an_em contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at /home2/cye73/biomedical-entity-linking/biomedical/bigbio/hub/hub_repos/an_em/an_em.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
/nethome/cye73/conda_envs/arboel_2/lib/python3.9/site-packages/datasets/load.py:923: FutureWarning: The repository for anat_em contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at /home2/cye73/biomedical-entity-linking/biomedical/bigbio/hub/hub_repos/anat_em/anat_em.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True`

In [2]:
umls_dir="/mitchell/entity-linking/2017AA/META/"


umls = UmlsMappings(
umls_dir=umls_dir, debug=False, force_reprocess=False,
)


Loading cached UMLS data from /mitchell/entity-linking/2017AA/META/.cached_df.feather


In [3]:
umls.umls.head()
# The .head() method in pandas is used to return the first N (=5 by default) rows of a DataFrame

,cui,lang,ispref,aui,scui,sdui,sab,tty,alias,suppress,rank,tui,group,def
0,C0000005,ENG,Y,A26634265,M0019694,D012711,MSH,PEP,(131)I-Macroaggregated Albumin,N,4,"[T116, T121, T130]",[CHEM],None
1,C0000074,ENG,Y,A26606894,M0016680,D010742,MSH,PEP,1-Alkyl-2-Acylphosphatidates,N,4,[T109],[CHEM],None
2,C0000132,ENG,Y,A26665454,M0004272,D002787,MSH,PEP,15-Ketosteryl Oleate Hydrolase,N,4,"[T116, T126]",[CHEM],None
3,C0000137,ENG,Y,A26650280,M0019173,D012335,MSH,PEP,15S RNA,N,4,"[T114, T123]",[CHEM],None
4,C0000151,ENG,Y,A26647507,M0020412,D013196,MSH,PEP,17 beta-Hydroxy-5 beta-Androstan-3-One,N,4,"[T109, T121]",[CHEM],None


In [4]:
umls_filtered = umls.filter_ontologies_and_types()
umls_filtered_df = pd.DataFrame(umls_filtered)
umls_filtered_df.head()

,cui,lang,ispref,aui,scui,sdui,sab,tty,alias,suppress,rank,tui,group,def
0,C0000005,ENG,Y,A26634265,M0019694,D012711,MSH,PEP,(131)I-Macroaggregated Albumin,N,4,"[T116, T121, T130]",[CHEM],None
1,C0000074,ENG,Y,A26606894,M0016680,D010742,MSH,PEP,1-Alkyl-2-Acylphosphatidates,N,4,[T109],[CHEM],None
2,C0000132,ENG,Y,A26665454,M0004272,D002787,MSH,PEP,15-Ketosteryl Oleate Hydrolase,N,4,"[T116, T126]",[CHEM],None
3,C0000137,ENG,Y,A26650280,M0019173,D012335,MSH,PEP,15S RNA,N,4,"[T114, T123]",[CHEM],None
4,C0000151,ENG,Y,A26647507,M0020412,D013196,MSH,PEP,17 beta-Hydroxy-5 beta-Androstan-3-One,N,4,"[T109, T121]",[CHEM],None


In [5]:
umls_cui2types = umls.umls.groupby('cui').tui.first().to_dict()
# Groups the rows of the umls DataFrame by the 'cui' column
# After grouping the DataFrame by 'cui', sort each group by ascending order of 'tui' (Type Unique Identifier)
# Converts the result into a dictionary


print('\nThe first 5 elements are :')
first_5_elements = list(umls_cui2types.items())[:5]
for cui, tui in first_5_elements:
    print(cui, tui)


The first 5 elements are :
C0000005 ['T116' 'T121' 'T130']
C0000039 ['T109' 'T121']
C0000052 ['T116' 'T126']
C0000074 ['T109']
C0000084 ['T116' 'T123']


In [6]:
lowercase=False

# Full UMLS Canonical Names
print("Getting Names")
umls_to_name = umls.get_canonical_name(ontologies_to_include="all",
    use_umls_curies=True,
    lowercase=lowercase,)

print("\nThis mapping is a specific application of the canonical name concept within the context of UMLS, linking each unique concept (identified by its CUI) to its preferred term")
print(list(umls_to_name.items())[:5])

Getting Names


/home2/cye73/biomedical-entity-linking/MyTests/../umls_utils.py:206: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["identifier"] = df["cui"]



This mapping is a specific application of the canonical name concept within the context of UMLS, linking each unique concept (identified by its CUI) to its preferred term
[('C0000005', '(131)I-Macroaggregated Albumin'), ('C0000039', '1,2-Dipalmitoylphosphatidylcholine'), ('C0000052', '1,4-alpha-Glucan Branching Enzyme'), ('C0000074', '1-Alkyl-2-Acylphosphatidates'), ('C0000084', '1-Carboxyglutamic Acid')]


In [7]:
all_umls_df = umls.umls.query('lang == "ENG"').groupby('cui').agg({'alias': lambda x: list(set(x)), 'tui':'first', 'group': 'first', 'def':'first'}).reset_index()
# filters rows where the column lang equals "ENG", indicating that only English language entries are considered for further operations.

# .reset_index() converts the index (in this case, the grouped CUIs) back into a regular column

all_umls_df['name'] = all_umls_df.cui.map(umls_to_name)
all_umls_df['alias'] = all_umls_df[['name','alias']].apply(lambda x: list(set(x[1]) - set([x[0]])) , axis=1)
all_umls_df['cui'] = all_umls_df['cui'].map(lambda x: 'UMLS' + x)
all_umls_df['has_definition'] = all_umls_df['def'].map(lambda x: x is not None)
all_umls_df['num_aliases'] = all_umls_df['alias'].map(lambda x: len(x))
all_umls_df.head()

/tmp/ipykernel_882210/2463801101.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  all_umls_df['alias'] = all_umls_df[['name','alias']].apply(lambda x: list(set(x[1]) - set([x[0]])) , axis=1)


,cui,alias,tui,group,def,name,has_definition,num_aliases
0,UMLSC0000005,[(131)I-MAA],"[T116, T121, T130]",[CHEM],None,(131)I-Macroaggregated Albumin,False,1
1,UMLSC0000039,"[3,5,9-Trioxa-4-phosphapentacosan-1-aminium, 4...","[T109, T121]",[CHEM],Synthetic phospholipid used in liposomes and l...,"1,2-Dipalmitoylphosphatidylcholine",True,15
2,UMLSC0000052,"[Branching Enzyme, Branching Enzyme, Starch, E...","[T116, T126]",[CHEM],"In glycogen or amylopectin synthesis, the enzy...","1,4-alpha-Glucan Branching Enzyme",True,22
3,UMLSC0000074,[1 Alkyl 2 Acylphosphatidates],[T109],[CHEM],None,1-Alkyl-2-Acylphosphatidates,False,1
4,UMLSC0000084,"[gamma Carboxyglutamic Acid, 1,1,3-Propanetric...","[T116, T123]",[CHEM],"Found in various tissues, particularly in four...",1-Carboxyglutamic Acid,True,7


In [8]:
print("Geting aliases")
umls_to_alias = umls.get_aliases(
    ontologies_to_include="all",
    use_umls_curies=True,
    lowercase=lowercase,
)

print('\nThe first 5 elements are :')
print(list(umls_to_alias.items())[:5])

Geting aliases

The first 5 elements are :
[('C0000005', ['(131)I-Macroaggregated Albumin', '(131)I-MAA']), ('C0000039', ['1,2-Dihexadecyl-sn-Glycerophosphocholine', '1,2-Dipalmitoyl-Glycerophosphocholine', 'Dipalmitoylphosphatidylcholine', 'Dipalmitoylglycerophosphocholine', 'Dipalmitoyllecithin', 'Dipalmitoyl Phosphatidylcholine', '1,2-Dipalmitoylphosphatidylcholine', '1,2 Dipalmitoylphosphatidylcholine', '1,2 Dihexadecyl sn Glycerophosphocholine', '1,2 Dipalmitoyl Glycerophosphocholine', 'Phosphatidylcholine, Dipalmitoyl', '3,5,9-Trioxa-4-phosphapentacosan-1-aminium, 4-hydroxy-N,N,N-trimethyl-10-oxo-7-((1-oxohexadecyl)oxy)-, inner salt, 4-oxide', 'Dipalmitoylphosphatidylcholine (substance)', 'DPPC', 'DIPALMITOYLPHOSPHATIDYLCHOLINE 0102', '1,2-Dipalmitoylphosphatidylcholine [Chemical/Ingredient]']), ('C0000052', ['Branching Enzyme', 'Branching Glycosyltransferase', 'Starch Branching Enzyme', '1,4-alpha-Glucan Branching Enzyme', '1,4 alpha Glucan Branching Enzyme', 'Branching Enzyme, 

In [9]:
# def get_type_gcd(types, type2geneology, cached_types: dict = None):
#     '''
#     Find the most granular single parent type for an entity with multiple types
#     If an entity is assigned multiple types with disjoint type hierarchies
#     '''
#     if len(types) == 1:
#         t = types[0]
#         if t not in cached_types:
#             cached_types[t] = type2geneology[t][-1]
#         return cached_types[t]

#     type_tuple = tuple(sorted(types))
#     if type_tuple in cached_types:
#         return cached_types[type_tuple]
#     else:
#         geneologies = [type2geneology[t] for t in types]
#         min_len = min([len(x) for x in geneologies])
#         arr = np.array([gen[:min_len] for gen in geneologies])
#         unique_types = np.unique(arr[:,0])
#         if len(unique_types) > 1:
#             output_types = ', '.join(list(set([x[-1] for x in geneologies])))
#             cached_types[type_tuple] = output_types
#             return output_types
#         else:
#             for i in np.arange(1, arr.shape[1]):
#                 curr_unique = np.unique(arr[:,i])

#                 if len(curr_unique) > 1 or i >= 3:
#                     cached_types[type_tuple] = unique_types[0]
#                     return unique_types[0]
#                 else:
#                     unique_types = curr_unique

In [10]:
# print("Getting types")
# type2geneology = ujson.load(open('../data/tui2type_hierarchy.json'))
# cached_types = {}
# cui2types = {}
# for cui, types in tqdm(umls_cui2types.items()):
#     cui2types[cui] = get_type_gcd(types, type2geneology, cached_types)

In [11]:
umls_cui2definition = umls.get_definition(
    ontologies_to_include="all",
    use_umls_curies=True,
    lowercase=False,
)

print('\nThe first 5 elements are:')
print(list(umls_cui2definition.items())[:5])

Filtering by ontologies
Removing null definitions

The first 5 elements are:
[('C0000039', 'Synthetic phospholipid used in liposomes and lipid bilayers to study biological membranes. It is also a major constituent of PULMONARY SURFACTANTS.'), ('C0000052', 'In glycogen or amylopectin synthesis, the enzyme that catalyzes the transfer of a segment of a 1,4-alpha-glucan chain to a primary hydroxy group in a similar glucan chain. EC 2.4.1.18.'), ('C0000084', 'Found in various tissues, particularly in four blood-clotting proteins including prothrombin, in kidney protein, in bone protein, and in the protein present in various ectopic calcifications.'), ('C0000096', 'A potent cyclic nucleotide phosphodiesterase inhibitor; due to this action, the compound increases cyclic AMP and cyclic GMP in tissue and thereby activates CYCLIC NUCLEOTIDE-REGULATED PROTEIN KINASES'), ('C0000097', 'A dopaminergic neurotoxic compound which produces irreversible clinical, chemical, and pathological alterations th

In [12]:
k = 0
for key, values in umls_cui2definition.items():
    print("CUI:", key, "definition:", values)
    k += 1
    if k >10 :
        break
    

CUI: C0000039 definition: Synthetic phospholipid used in liposomes and lipid bilayers to study biological membranes. It is also a major constituent of PULMONARY SURFACTANTS.
CUI: C0000052 definition: In glycogen or amylopectin synthesis, the enzyme that catalyzes the transfer of a segment of a 1,4-alpha-glucan chain to a primary hydroxy group in a similar glucan chain. EC 2.4.1.18.
CUI: C0000084 definition: Found in various tissues, particularly in four blood-clotting proteins including prothrombin, in kidney protein, in bone protein, and in the protein present in various ectopic calcifications.
CUI: C0000096 definition: A potent cyclic nucleotide phosphodiesterase inhibitor; due to this action, the compound increases cyclic AMP and cyclic GMP in tissue and thereby activates CYCLIC NUCLEOTIDE-REGULATED PROTEIN KINASES
CUI: C0000097 definition: A dopaminergic neurotoxic compound which produces irreversible clinical, chemical, and pathological alterations that mimic those found in Parkin

In [13]:
import os
print(os.getcwd())

/home2/cye73/biomedical-entity-linking/MyTests


In [14]:
# Check if the directory exists, and create it if it does not
name = 'medmentions_full'
directory_path = f'../data/arboel/{name}/' 
if not os.path.exists(directory_path):
    os.makedirs(directory_path)

umls_entities = []
for cui, name in tqdm(umls_to_name.items()):
    d = {}
    ent_type = umls_cui2types[cui][0]
    other_aliases = [x for x in umls_to_alias[cui] if x != name]
    joined_aliases = ' ; '.join(other_aliases)
    d['cui'] = f"UMLS:{cui}"
    d['title'] = name
    d['type'] = ent_type
    if cui in umls_cui2definition:
        definition = umls_cui2definition[cui]
    else:
        definition = None

    if len(other_aliases) > 0:
        if definition is not None:
            d['description'] = f"{name} ( {ent_type} : {joined_aliases} ) [ {definition} ]"
        else:
            d['description'] = f"{name} ( {ent_type} : {joined_aliases} )"
    else:
        if definition is not None:
            d['description'] = f"{name} ( {ent_type} ) [ {definition} ]"
        else:
            d['description'] = f"{name} ( {ent_type} )"

    umls_entities.append(d)

# Created in "/home2/cye73/data/arboel/medmentions_full//dictionary.pickle"
# pickle.dump(umls_entities, open('../data/arboel/medmentions_full/dictionary.pickle', 'wb'))

'''
Example of elements in umls_entities
{
    'cui': 'UMLS:C0000005',
    'title': '(131)I-Macroaggregated Albumin',
    'type': 'ChemicalSubstance',
    'description': '(131)I-Macroaggregated Albumin ( ChemicalSubstance : Alias1 ; Alias2 ) [ A radioactive diagnostic agent used in pulmonary imaging. ]'
}
'''



100%|██████████| 3465007/3465007 [00:11<00:00, 314249.25it/s]


"\nExample of elements in umls_entities\n{\n    'cui': 'UMLS:C0000005',\n    'title': '(131)I-Macroaggregated Albumin',\n    'type': 'ChemicalSubstance',\n    'description': '(131)I-Macroaggregated Albumin ( ChemicalSubstance : Alias1 ; Alias2 ) [ A radioactive diagnostic agent used in pulmonary imaging. ]'\n}\n"

In [15]:
umls_entities[0]

{'cui': 'UMLS:C0000005',
 'title': '(131)I-Macroaggregated Albumin',
 'type': 'T116',
 'description': '(131)I-Macroaggregated Albumin ( T116 : (131)I-MAA )'}

In [17]:
import sys
from tqdm import tqdm
# Append the directory to sys.path
sys.path.append('/home2/cye73/arboEL')

from pytorch_transformers.tokenization_bert import BertTokenizer
from blink.biencoder.data_process_mult import get_candidate_representation

In [18]:
# Add "tokens" and "ids" keys
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
dict_cui_to_idx = {}
dictionary_processed = False
max_cand_length = 64
for idx, ent in enumerate(tqdm(umls_entities[:3], desc="Tokenizing dictionary")):
    dict_cui_to_idx[ent["cui"]] = idx
    if not dictionary_processed:
        label_representation = get_candidate_representation(
            ent["description"], tokenizer, max_cand_length, ent["title"]
        )
        umls_entities[idx]["tokens"] = label_representation["tokens"]
        umls_entities[idx]["ids"] = label_representation["ids"]

Tokenizing dictionary: 100%|██████████| 3/3 [00:00<00:00, 542.20it/s]


In [19]:
umls_entities[0]['description']

'(131)I-Macroaggregated Albumin ( T116 : (131)I-MAA )'

In [20]:
umls_entities[0]['tokens']

['[CLS]',
 '(',
 '131',
 ')',
 'i',
 '-',
 'macro',
 '##ag',
 '##gre',
 '##gated',
 'album',
 '##in',
 '[unused3]',
 '(',
 't',
 '##11',
 '##6',
 ':',
 '(',
 '131',
 ')',
 'i',
 '-',
 'ma',
 '##a',
 ')',
 '[SEP]']

# Function for processing umls ontology

In [3]:
import os
import pickle
from tqdm import tqdm

def process_umls_ontology(ontology, data_path, ontology_dir):
    '''
    This function prepares the entity data : dictionary.pickle
    
    Parameters 
    ----------
    - ontology : str (only umls for now)
        Ontology associated with the dataset
    - data_path : str
        Path where to load and save dictionary.pickle
    - ontology_dir : str
        Path to UMLS ontology
    '''
    
    ontology = BiomedicalOntology(name="UMLS")
    ontology.load_umls(path = ontology_dir)
    
    equivalent_cuis = False
    if ontology.entities[0].equivalant_cuis is not None : 
        equivalent_cuis = True
        
    'If dictionary already processed, load it else process and load it'
    entity_dictionary_pkl_path = os.path.join(data_path, 'dictionary.pickle')
    
    if os.path.isfile(entity_dictionary_pkl_path): 
        print("Loading stored processed entity dictionary...")
        with open(entity_dictionary_pkl_path, 'rb') as read_handle:
            entities = pickle.load(read_handle)
        
        return entities, equivalent_cuis
    
    ontology_entities = []
    for entity in tqdm(ontology.entities):      
        if entity.aliases != "":
            if entity.definition is not None:
                new_entity = {
                    'type': entity.types[0],
                    'cui': entity.cui,
                    'title': entity.name,
                    'description': f"{entity.name} ( {entity.types[0]} : {entity.aliases} ) [{entity.definition}]"
                }
            else : 
                new_entity = {
                    'type': entity.types[0],
                    'cui': entity.cui,
                    'title': entity.name,
                    'description': f"{entity.name} ( {entity.types[0]} : {entity.aliases} )"
                }
                
        else : 
            if entity.definition is not None:
                new_entity = {
                        'type': entity.types[0],
                        'cui': entity.cui,
                        'title': entity.name,
                        'description': f"{entity.name} ( {entity.types[0]} ) [{entity.definition}]"
                    }
            else : 
                new_entity = {
                        'type': entity.types[0],
                        'cui': entity.cui,
                        'title': entity.name,
                        'description': f"{entity.name} ( {entity.types[0]} )"
                    }
        ontology_entities.append(new_entity)

    # Check if the directory exists, and create it if it does not
    if not os.path.exists(data_path):
        os.makedirs(data_path)
        
    # Save entities to pickle file
    with open(os.path.join(data_path, 'dictionary.pickle'), 'wb') as f:
        pickle.dump(ontology_entities, f)
        
    return ontology_entities, equivalent_cuis



In [27]:
ontology = BiomedicalOntology(name="UMLS")
ontology.load_umls(path = ontology_dir)

Loading cached UMLS data from /mitchell/entity-linking/2017AA/META/.cached_df.feather


/home2/cye73/biomedical-entity-linking/arboEL/blink/biencoder/MyDataModuleTests/../../../../umls_utils.py:206: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["identifier"] = df["cui"]
/home2/cye73/biomedical-entity-linking/arboEL/blink/biencoder/MyDataModuleTests/../../../../bioel/ontology.py:248: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  lambda x: list(set(x[1]) - set([x[0]])), axis=1


In [28]:
ontology.entities[0]

BiomedicalEntity(cui='UMLS:C0000005', name='(131)I-Macroaggregated Albumin', types=array(['T116', 'T121', 'T130'], dtype=object), aliases=['(131)I-MAA'], definition=None, equivalant_cuis=None, taxonomy=None, metadata=None, extra_data={'group': array(['CHEM'], dtype=object)})

## Test if the function works

In [4]:
ontology_dir = "/mitchell/entity-linking/2017AA/META/"
ontology = "umls"
model = "arboel"
dataset = "medmentions_st21pv"
abs_path = "/home2/cye73/data"
data_path = os.path.join(abs_path, model, dataset)
print(data_path)

entities = process_umls_ontology(ontology = ontology,
                                    data_path= data_path,
                                    ontology_dir = ontology_dir, )


/home2/cye73/data/arboel/medmentions_st21pv
Loading cached UMLS data from /mitchell/entity-linking/2017AA/META/.cached_df.feather


/home2/cye73/biomedical-entity-linking/arboEL/blink/biencoder/MyDataModuleTests/../../../../umls_utils.py:206: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["identifier"] = df["cui"]
/home2/cye73/biomedical-entity-linking/arboEL/blink/biencoder/MyDataModuleTests/../../../../bioel/ontology.py:248: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  lambda x: list(set(x[1]) - set([x[0]])), axis=1


Loading stored processed entity dictionary...


In [8]:
entities[0][0]

{'type': 'T116',
 'cui': 'UMLS:C0000005',
 'title': '(131)I-Macroaggregated Albumin',
 'description': "(131)I-Macroaggregated Albumin ( T116 : ['(131)I-MAA'] )"}

# Function for processing dataset linked to umls

In [11]:
def process_mention_dataset(
    ontology,
    dataset,
    data_path,
    obo_dict: Optional[dict] = None,
    ontology_dir: Optional[str] = None,
    mention_id: Optional[bool] = True,
    context_doc_id: Optional[bool] = True,
):
    """
    This function prepares the mentions data :  Creates the train.jsonl, valid.jsonl, test.jsonl
    Each .jsonl contains data in the following format :
    {'mention': mention,
    'mention_id': ID of the mention, (optional)
    'context_left': context before mention,
    'context_right': context after mention,
    'context_doc_id': ID of the doc, (optional)
    'type': type
    'label_id': label_id,
    'label': entity description, (optional)
    'label_title': entity title

    Parameters
    ----------
    - ontology : str
    Ontology associated with the dataset
    - dataset : str
    Name of the dataset
    - data_path : str
    Path where to load and save dictionary.pickle
    - ontology_dir : str
    Path to the ontology (umls, medic etc...)
    """
    data = conhelps.for_config_name(f"{dataset}_bigbio_kb").load_dataset()
    exclude = CUIS_TO_EXCLUDE[dataset]
    remap = CUIS_TO_REMAP[dataset]

    if ontology == "obo":
        entities, equivalant_cuis = process_obo_ontology(ontology, data_path, obo_dict)
    elif ontology == "medic":
        entities, equivalant_cuis = process_medic_ontology(ontology, data_path, ontology_dir)
    elif ontology == "umls":
        entities, equivalant_cuis = process_umls_ontology(ontology, data_path, ontology_dir)
    else:
        print("ERROR!")

    entity_dictionary = {d["cui"]: d for d in tqdm(entities)}  # CC1

    "For ontology with multiples cuis"
    if equivalant_cuis :  # CC2
        # Need to redo this since we have multiple synonymous CUIs for ncbi_disease
        entity_dictionary = {cui: d for d in tqdm(entities) for cui in d["cuis"]}
        cui_synsets = {}
        for subdict in tqdm(entities):
            for cui in subdict["cuis"]:
                if cui in subdict:
                    print(cui, cui_synsets[cui], subdict["cuis"])
                cui_synsets[cui] = subdict["cuis"]
        with open(os.path.join(data_path, "cui_synsets.json"), "w") as f:
            f.write(ujson.dumps(cui_synsets, indent=2))

    if dataset in VALIDATION_DOCUMENT_IDS:
        validation_pmids = VALIDATION_DOCUMENT_IDS[dataset]
    else:
        print("ERROR!!!")

    # Convert BigBio dataset to pandas DataFrame
    df = dataset_to_df(
        data,
        entity_remapping_dict=remap,
        cuis_to_exclude=exclude,
        val_split_ids=validation_pmids,
    )
    # Return dictionary of documents in BigBio dataset
    docs = dataset_to_documents(data)
    label_len = df["db_ids"].map(lambda x: len(x)).max()
    print("Max labels on one doc:", label_len)

    for split in df.split.unique():
        ents_in_split = []
        for d in tqdm(
            df.query("split == @split").to_dict(orient="records"),
            desc=f"Creating correct mention format for {split} dataset",
        ):
            abbrev_resolved = False
            offsets = d["offsets"]
            doc_id = d["document_id"]
            doc = docs[doc_id]
            mention = d["text"]

            # Get offsets and context
            start = offsets[0][0]  # start on the mention
            end = offsets[-1][-1]  # end of the mention
            before_context = doc[:start]  # left context
            after_context = doc[end:]  # right context

            # ArboEL can't handle multi-labels, so we randomly choose one.
            if len(d["db_ids"]) == 1:
                label_id = d["db_ids"][0]

            # ncbi_disease is a special case that requires extra care
            elif equivalant_cuis :
                labels = []
                used_cuis = set([])
                choosable_ids = []
                for db_id in d["db_ids"]:
                    if db_id in used_cuis:
                        continue
                    else:
                        used_cuis.update(set(entity_dictionary[db_id]["cuis"]))
                    choosable_ids.append(db_id)

                label_id = np.random.choice(choosable_ids)

            else:
                label_id = np.random.choice(d["db_ids"])

            # Check if we missed something
            if label_id not in entity_dictionary:
                # print(label_id)
                continue
            
            output = [
                {
                    "mention": mention,
                    "context_left": before_context,
                    "context_right": after_context,
                    "type": d["type"][0],
                    "label_id": label_id,
                    "label_title": entity_dictionary[label_id]["title"],
                    "label": entity_dictionary[label_id]["description"],
                }
            ]

            if mention_id:
                output[0]["mention_id"] = d.get("mention_id", None)

            if context_doc_id:
                output[0]["context_doc_id"] = d.get("document_id", None)

            ents_in_split.extend(output)

        split_name = split
        if split == "validation":
            split_name = "valid"
        with open(os.path.join(data_path, f"{split_name}.jsonl"), "w") as f:
            f.write("\n".join([ujson.dumps(x) for x in ents_in_split]))
    return output

In [12]:
mentions = process_mention_dataset(ontology = ontology,
                                    dataset = dataset,
                                    data_path = data_path,
                                    ontology_dir = ontology_dir,
                                    mention_id = True,
                                    context_doc_id = True,
                                    )



Loading cached UMLS data from /mitchell/entity-linking/2017AA/META/.cached_df.feather


/home2/cye73/biomedical-entity-linking/arboEL/blink/biencoder/MyDataModuleTests/../../../../umls_utils.py:206: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["identifier"] = df["cui"]
/home2/cye73/biomedical-entity-linking/arboEL/blink/biencoder/MyDataModuleTests/../../../../bioel/ontology.py:248: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  lambda x: list(set(x[1]) - set([x[0]])), axis=1


Loading stored processed entity dictionary...


100%|██████████| 3465007/3465007 [00:01<00:00, 1856395.18it/s]


Max labels on one doc: 2


Creating correct mention format for train dataset: 100%|██████████| 122174/122174 [00:00<00:00, 249032.85it/s]
Creating correct mention format for test dataset: 100%|██████████| 40143/40143 [00:00<00:00, 302844.67it/s]
Creating correct mention format for validation dataset: 100%|██████████| 40864/40864 [00:00<00:00, 304827.63it/s]


In [13]:
mentions[0]

{'mention': 'hospitalisation',
 'context_left': 'Medication regimen complexity and prevalence of potentially inappropriate medicines in older patients after hospitalisation\nBackground There is a relative paucity of information to characterise potential changes in medication regimen complexity and prevalence of prescribing of potentially inappropriate medications after hospitalisation, both in Australia and elsewhere. Objective To evaluate medication regimen complexity and the prevalence of potentially inappropriate medications before and after admission to hospital. Setting General medical units of a tertiary care hospital in Australia. Methods Retrospective cohort study of patients aged 65 years and above. Medication complexity was measured by using the Medication Regimen Complexity Index (MRCI). Main outcome measure The primary outcome was the change in the Medication Regimen Complexity Index for all prescribed medications after hospitalization. Results A convenience sample of 100 p

## INCLUDE CUI2TYPES

In [ ]:
umls_entities = []
for cui, name in tqdm(umls_to_name.items()):
    d = {}
    ent_type = cui2types[cui]
    other_aliases = [x for x in umls_to_alias[cui] if x != name]
    joined_aliases = ' ; '.join(other_aliases)
    d['cui'] = f"UMLS:{cui}"
    d['title'] = name
    d['type'] = ent_type
    if cui in umls_cui2definition:
        definition = umls_cui2definition[cui]
    else:
        definition = None

    if len(other_aliases) > 0:
        if definition is not None:
            d['description'] = f"{name} ( {ent_type} : {joined_aliases} ) [ {definition} ]"
        else:
            d['description'] = f"{name} ( {ent_type} : {joined_aliases} )"
    else:
        if definition is not None:
            d['description'] = f"{name} ( {ent_type} ) [ {definition} ]"
        else:
            d['description'] = f"{name} ( {ent_type} )"

    umls_entities.append(d)

pickle.dump(umls_entities, open('../data/arboel/medmentions_full/dictionary.pickle', 'wb'))

'''
Example of elements in umls_entities
{
    'cui': 'UMLS:C0000005',
    'title': '(131)I-Macroaggregated Albumin',
    'type': 'ChemicalSubstance',
    'description': '(131)I-Macroaggregated Albumin ( ChemicalSubstance : Alias1 ; Alias2 ) [ A radioactive diagnostic agent used in pulmonary imaging. ]'
}
'''

## Example

In [26]:
# Check if the directory exists, and create it if it does not
name = 'medmentions_full'
directory_path = f'../data/arboel/{name}/' 
if not os.path.exists(directory_path):
    os.makedirs(directory_path)

umls_entities = []
for cui, name in tqdm(umls_to_name.items()):
    d = {}
    # ent_type = cui2types[cui]
    other_aliases = [x for x in umls_to_alias[cui] if x != name]
    joined_aliases = ' ; '.join(other_aliases)
    d['cui'] = f"UMLS:{cui}"
    d['title'] = name
    # d['type'] = ent_type
    if cui in umls_cui2definition:
        definition = umls_cui2definition[cui]
    else:
        definition = None

    if len(other_aliases) > 0:
        if definition is not None:
            d['description'] = f"{name} ({joined_aliases}) [ {definition} ]"
        else:
            d['description'] = f"{name} ({joined_aliases})"
    else:
        if definition is not None:
            d['description'] = f"{name} [ {definition} ]"
        else:
            d['description'] = f"{name}"

    umls_entities.append(d)


'''
Example of elements in umls_entities
{
    'cui': 'UMLS:C0000005',
    'title': '(131)I-Macroaggregated Albumin',
    'type': 'ChemicalSubstance',
    'description': '(131)I-Macroaggregated Albumin ( ChemicalSubstance : Alias1 ; Alias2 ) [ A radioactive diagnostic agent used in pulmonary imaging. ]'
}
'''



100%|██████████| 3465007/3465007 [00:07<00:00, 473224.98it/s]


"\nExample of elements in umls_entities\n{\n    'cui': 'UMLS:C0000005',\n    'title': '(131)I-Macroaggregated Albumin',\n    'type': 'ChemicalSubstance',\n    'description': '(131)I-Macroaggregated Albumin ( ChemicalSubstance : Alias1 ; Alias2 ) [ A radioactive diagnostic agent used in pulmonary imaging. ]'\n}\n"

In [ ]:
# Function for retrieving the title + description of entities of umls_ontology
def process_umls_ontology(ontology, data_path, umls_dir):  # not used
    """
    This function prepares the entity data : dictionary.pickle

    Parameters
    ----------
    - ontology : str
    Ontology associated with the dataset
    - data_path : str
    Path where to load and save dictionary.pickle
    - umls_dir : str
    Path to umls data
    """
    umls = UmlsMappings(
        umls_dir=umls_dir,
        debug=False,
        force_reprocess=False,
    )

    umls_to_name = umls.get_canonical_name(
        ontologies_to_include="all", use_umls_curies=True, lowercase=False
    )

    umls_to_alias = umls.get_aliases(
        ontologies_to_include="all", use_umls_curies=True, lowercase=False
    )

    umls_cui2definition = umls.get_definition(
        ontologies_to_include="all", use_umls_curies=True, lowercase=False
    )

    umls_cui2types = umls.umls.groupby("cui").tui.first().to_dict()

    # Check if the directory exists, and create it if it does not
    if not os.path.exists(data_path):
        os.makedirs(data_path)

    umls_entities = []
    for cui, name in tqdm(umls_to_name.items(), "Creating correct entities format"):
        d = {}
        ent_type = umls_cui2types[cui][0]
        other_aliases = [x for x in umls_to_alias[cui] if x != name]
        joined_aliases = " ; ".join(other_aliases)
        d["cui"] = f"UMLS:{cui}"
        d["title"] = name
        d["type"] = ent_type
        if cui in umls_cui2definition:
            definition = umls_cui2definition[cui]
        else:
            definition = None

        if len(other_aliases) > 0:
            if definition is not None:
                d["description"] = (
                    f"{name} ( {ent_type} : {joined_aliases} ) [ {definition} ]"
                )
            else:
                d["description"] = f"{name} ( {ent_type} : {joined_aliases} )"
        else:
            if definition is not None:
                d["description"] = f"{name} ( {ent_type} ) [ {definition} ]"
            else:
                d["description"] = f"{name} ( {ent_type} )"

        umls_entities.append(d)

    pickle.dump(umls_entities, open(os.path.join(data_path, "dictionary.pickle"), "wb"))
    entities = pickle.load(open(os.path.join(data_path, "dictionary.pickle"), "rb"))
    return entities


# Function for preparing the mentions in the dataset into the right format for each model


